In [1]:
cd /Users/Liangqiaohao/Desktop/PV Lab/2020/March/Learning project/C2a

/Users/Liangqiaohao/Desktop/PV Lab/2020/March/Learning project/C2a


In [2]:
import numpy as np
import torch
import math
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import GPyOpt

import os

import matplotlib as mpl
import matplotlib.tri as tri

import ternary
import pickle
import datetime

from collections import Counter
import matplotlib.ticker as ticker

import pyDOE
import random
from scipy.stats import norm

from sklearn.ensemble import RandomForestRegressor

In [3]:
dataset = pd.read_excel(r'/Users/Liangqiaohao/Desktop/PV Lab/2020/March/Learning project/C2a/C2a learning study.xlsx')
X = dataset.iloc[:,0:5]

X_run = X.groupby(['CsPbI', 'MAPbI', 'FAPbI'])['Instability index'].agg(lambda x: x.unique().mean())
X_run = (X_run.to_frame()).reset_index()

materials = ['CsPbI', 'MAPbI', 'FAPbI']
X_feature = X_run[['CsPbI', 'MAPbI', 'FAPbI']].values

X_Cs = np.array(X_run['CsPbI'].values)
X_FA = np.array(X_run['FAPbI'].values)
X_MA = np.array(X_run['MAPbI'].values)
X_instability = np.array(X_run['Instability index'].values)

# if ever use for round study
# X_r0 = X_run[X['Round'] == 0]
# X_r1 = X_run[X['Round'] == 1]
# X_r2 = X_run[X['Round'] == 2]
# X_r3 = X_run[X['Round'] == 3]


X_run


# hide_toggle('load data')

,CsPbI,MAPbI,FAPbI,Instability index
0,0.00,0.00,1.00,4.929210e+05
1,0.00,0.25,0.75,9.480030e+05
2,0.00,0.50,0.50,1.129496e+06
3,0.00,0.75,0.25,1.290847e+06
4,0.00,1.00,0.00,1.452513e+06
...,...,...,...,...
89,0.88,0.12,0.00,4.048450e+05
90,0.89,0.11,0.00,4.819935e+05
91,0.91,0.08,0.01,4.822180e+05
92,0.96,0.00,0.04,3.497070e+05


In [4]:
bounds = [None for j in range(len(materials))]
for j in range(len(materials)):
    bounds[j] = {'name': materials[j], 'type': 'continuous', 'domain': (0,1)}

tolerance_factor_constraint = '((167*x[:,0]+217*x[:,1]+253*x[:,2])+220)/(1.4142*(119+220))'
constraints_baseline = [{'name': 'constr_A_max', 'constraint': 'x[:,0] +x[:,1] +x[:,2]- ' + str(1)},
                        {'name': 'constr_A_min', 'constraint': str(0.99)+'-x[:,0] -x[:,1]-x[:,2]'},
                        {'name': 'G_tolerance_min', 'constraint': '0.80 - ' + tolerance_factor_constraint},
                        {'name': 'G_tolerance_max', 'constraint': tolerance_factor_constraint + ' - 1'}
                       ]


In [64]:

# total 94 pts
n_initial = 5
n_test = int(np.round(len(X_instability) * 0.1))
jitter = 0.01

top5_indices = list(X_run.sort_values('Instability index').head(5).index)

seed_list = [74,660,861,1526,2,151,19,7,33,4021,66,42424,671,25,95711,850,71,2148,26219,4]
# 20

n_estimators = 50

In [6]:


def EI(X, model, y_best, input_mean, input_std):
#     xi = 0.01
    normalized_mean, normalized_std = model.predict(X)[0][0][0], model.predict(X)[1][0][0]
    
    mean = normalized_mean * input_std + input_mean
    std = normalized_std * input_std
    
#     z = (mean - y_best - xi)/std   "this is for maximization"
#     return (mean - y_best - xi) * norm.cdf(z) + std * norm.pdf(z)
#     z = (y_best - mean + xi)/std
#     return (y_best - mean + xi) * norm.cdf(z) + std * norm.pdf(z)
    z = (y_best - mean)/std
    return (y_best - mean) * norm.cdf(z) + std * norm.pdf(z)

def LCB(X, model, input_mean, input_std):
    normalized_mean, normalized_std = model.predict(X)[0][0][0], model.predict(X)[1][0][0]
    mean = normalized_mean * input_std + input_mean
    std = normalized_std * input_std
    
    return 2 * std - mean

def MPI(X, model, y_best, input_mean, input_std):
    normalized_mean, normalized_std = model.predict(X)[0][0][0], model.predict(X)[1][0][0]
    
    mean = normalized_mean * input_std + input_mean
    std = normalized_std * input_std
    
    z = (y_best - mean)/std
    return norm.cdf(z)

def calc_MAE(test_index, model, input_mean, input_std):
    total_mae = 0
    for index in test_index:
        X = X_feature[index]
        y = X_instability[index]
        normalized_mean, normalized_std = model.predict(X)[0][0][0], model.predict(X)[1][0][0]
        mean = normalized_mean * input_std + input_mean
        std = normalized_std * input_std
        mae_ = (y - mean)**2
        total_mae += mae_
    return total_mae / len(test_index)

def Top5Percent(top5_index, index_):
    how_many = 0
    for i in index_:
        if i in top5_index:
            how_many += 1
    return how_many
 


In [19]:
index_collection = []
X_collection = []
Instability_collection = []
InTopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    indices = list(np.arange(len(X_run)))
    
    top5_index = top5_indices
    X_top5 = []
    Instability_top5 = []
    for i in top5_index:
        X_top5.append(X_feature[i])
        Instability_top5.append(X_instability[i])
    
    index_ = random.sample(indices, n_initial)
    X_ = []
    Instability_ = []
    InTopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        Instability_.append(X_instability[i])
        indices.remove(i)
    
    test_index = random.sample(indices, n_test)
    X_test = []
    Instability_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        Instability_test.append(X_instability[i])
        indices.remove(i)
    
    n_opt = len(indices)
    assert n_opt == len(X_run) - n_test - n_initial
    
    for i in np.arange(n_opt):
        
        input_mean = np.mean(Instability_)
        input_std = np.std(Instability_)
        input_best = np.min(Instability_)
        
        BO_model = GPyOpt.methods.BayesianOptimization(f=None,  
                                                   domain = bounds,
                                                   constraints = constraints_baseline,
                                                   acquisition_type = 'EI',              
                                                   normalize_Y = True,
                                                   X = np.array(X_),
                                                   Y = np.array([[i] for i in Instability_]),
                                                   evaluator_type = 'local_penalization',
                                                   batch_size = 1,
                                                   acquisition_jitter = jitter)

        _ = BO_model.suggest_next_locations()
        
        next_index = None
        max_ac = -10**10
        for j in indices:
            X_j = X_feature[j]
            Instability_j = X_instability[j]
            
            ac_value = EI(X_j, BO_model.model, input_best, input_mean, input_std)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        Instability_.append(X_instability[next_index])
        indices.remove(next_index)
        index_.append(next_index)
        mae = calc_MAE(test_index, BO_model.model, input_mean, input_std)
        MAE_.append(mae)
        how_many = Top5Percent(top5_index, index_)
        InTopPercent_.append(how_many)
        
#         print('Finished '+str(len(MAE_)) + ' round')
        
    assert len(MAE_) == len(X_) - n_initial

    index_collection.append(index_)
    X_collection.append(X_)
    Instability_collection.append(Instability_)
    InTopPercent_collection.append(InTopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, Instability_collection, InTopPercent_collection, MAE_collection])
np.save('BO_EI_master', master)
   

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed


In [20]:
index_collection = []
X_collection = []
Instability_collection = []
InTopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    indices = list(np.arange(len(X_run)))
    
    top5_index = top5_indices
    X_top5 = []
    Instability_top5 = []
    for i in top5_index:
        X_top5.append(X_feature[i])
        Instability_top5.append(X_instability[i])
    
    index_ = random.sample(indices, n_initial)
    X_ = []
    Instability_ = []
    InTopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        Instability_.append(X_instability[i])
        indices.remove(i)
    
    test_index = random.sample(indices, n_test)
    X_test = []
    Instability_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        Instability_test.append(X_instability[i])
        indices.remove(i)
    
    n_opt = len(indices)
    assert n_opt == len(X_run) - n_test - n_initial
    
    for i in np.arange(n_opt):
        
        input_mean = np.mean(Instability_)
        input_std = np.std(Instability_)
        input_best = np.min(Instability_)
        
        BO_model = GPyOpt.methods.BayesianOptimization(f=None,  
                                                   domain = bounds,
                                                   constraints = constraints_baseline,
                                                   acquisition_type = 'LCB',              
                                                   normalize_Y = True,
                                                   X = np.array(X_),
                                                   Y = np.array([[i] for i in Instability_]),
                                                   evaluator_type = 'local_penalization',
                                                   batch_size = 1,
                                                   acquisition_jitter = jitter)

        _ = BO_model.suggest_next_locations()
        
        next_index = None
        max_ac = -10**10
        for j in indices:
            X_j = X_feature[j]
            Instability_j = X_instability[j]
            
            ac_value = LCB(X_j, BO_model.model, input_mean, input_std)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        Instability_.append(X_instability[next_index])
        indices.remove(next_index)
        index_.append(next_index)
        mae = calc_MAE(test_index, BO_model.model, input_mean, input_std)
        MAE_.append(mae)
        how_many = Top5Percent(top5_index, index_)
        InTopPercent_.append(how_many)
        
#         print('Finished '+str(len(MAE_)) + ' round')
        
    assert len(MAE_) == len(X_) - n_initial

    index_collection.append(index_)
    X_collection.append(X_)
    Instability_collection.append(Instability_)
    InTopPercent_collection.append(InTopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, Instability_collection, InTopPercent_collection, MAE_collection])
np.save('BO_LCB_master', master)

initializing seed = 0
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The se

In [21]:
index_collection = []
X_collection = []
Instability_collection = []
InTopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    indices = list(np.arange(len(X_run)))
    
    top5_index = top5_indices
    X_top5 = []
    Instability_top5 = []
    for i in top5_index:
        X_top5.append(X_feature[i])
        Instability_top5.append(X_instability[i])
    
    index_ = random.sample(indices, n_initial)
    X_ = []
    Instability_ = []
    InTopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        Instability_.append(X_instability[i])
        indices.remove(i)
    
    test_index = random.sample(indices, n_test)
    X_test = []
    Instability_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        Instability_test.append(X_instability[i])
        indices.remove(i)
    
    n_opt = len(indices)
    assert n_opt == len(X_run) - n_test - n_initial
    
    for i in np.arange(n_opt):
        
        input_mean = np.mean(Instability_)
        input_std = np.std(Instability_)
        input_best = np.min(Instability_)
        
        BO_model = GPyOpt.methods.BayesianOptimization(f=None,  
                                                   domain = bounds,
                                                   constraints = constraints_baseline,
                                                   acquisition_type = 'MPI',              
                                                   normalize_Y = True,
                                                   X = np.array(X_),
                                                   Y = np.array([[i] for i in Instability_]),
                                                   evaluator_type = 'local_penalization',
                                                   batch_size = 1,
                                                   acquisition_jitter = jitter)

        _ = BO_model.suggest_next_locations()
        
        next_index = None
        max_ac = -10**10
        for j in indices:
            X_j = X_feature[j]
            Instability_j = X_instability[j]
            
            ac_value = MPI(X_j, BO_model.model, input_best, input_mean, input_std)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        Instability_.append(X_instability[next_index])
        indices.remove(next_index)
        index_.append(next_index)
        mae = calc_MAE(test_index, BO_model.model, input_mean, input_std)
        MAE_.append(mae)
        how_many = Top5Percent(top5_index, index_)
        InTopPercent_.append(how_many)
        
#         print('Finished '+str(len(MAE_)) + ' round')
        
    assert len(MAE_) == len(X_) - n_initial

    index_collection.append(index_)
    X_collection.append(X_)
    Instability_collection.append(Instability_)
    InTopPercent_collection.append(InTopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, Instability_collection, InTopPercent_collection, MAE_collection])
np.save('BO_MPI_master', master)

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed


In [22]:
# random sampling

index_collection = []
X_collection = []
Instability_collection = []
InTopPercent_collection = []
MAE_collection = []

for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    indices = list(np.arange(len(X_run)))
    
    top5_index = top5_indices
    X_top5 = []
    Instability_top5 = []
    for i in top5_index:
        X_top5.append(X_feature[i])
        Instability_top5.append(X_instability[i])
    
    index_ = random.sample(indices, n_initial)
    X_ = []
    Instability_ = []
    InTopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        Instability_.append(X_instability[i])
        indices.remove(i)
    
    test_index = random.sample(indices, n_test)
    X_test = []
    Instability_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        Instability_test.append(X_instability[i])
        indices.remove(i)
    
    n_opt = len(indices)
    assert n_opt == len(X_run) - n_test - n_initial

    for i in np.arange(n_opt):
    
        input_mean = np.mean(Instability_)
        input_std = np.std(Instability_)
        input_best = np.min(Instability_)
        
        BO_model = GPyOpt.methods.BayesianOptimization(f=None,  
                                                   domain = bounds,
                                                   constraints = constraints_baseline,
                                                   acquisition_type = 'EI',              
                                                   normalize_Y = True,
                                                   X = np.array(X_),
                                                   Y = np.array([[i] for i in Instability_]),
                                                   evaluator_type = 'local_penalization',
                                                   batch_size = 1,
                                                   acquisition_jitter = jitter)

        _ = BO_model.suggest_next_locations()
        
        next_index = (random.sample(indices, 1))[0]
                
        X_.append(X_feature[next_index])
        Instability_.append(X_instability[next_index])
        indices.remove(next_index)
        index_.append(next_index)
        mae = calc_MAE(test_index, BO_model.model, input_mean, input_std)
        MAE_.append(mae)
        how_many = Top5Percent(top5_index, index_)
        InTopPercent_.append(how_many)
        
#         print('Finished '+str(len(MAE_)) + ' round')
        
    assert len(MAE_) == len(X_) - n_initial

    index_collection.append(index_)
    X_collection.append(X_)
    Instability_collection.append(Instability_)
    InTopPercent_collection.append(InTopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
master = np.array([index_collection, X_collection, Instability_collection, InTopPercent_collection, MAE_collection])
np.save('RS_master', master)

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed


In [82]:
def EI(X, RF_model, y_best):

    
    tree_predictions = []
    for j in np.arange(n_estimators):
        tree_predictions.append((RF_model.estimators_[j].predict(np.array([X]))).tolist())
    mean = np.mean(np.array(tree_predictions), axis=0)[0]

    
    std = np.std(np.array(tree_predictions), axis=0)[0]
    
    z = (y_best - mean)/std
    return (y_best - mean) * norm.cdf(z) + std * norm.pdf(z)

def LCB(X, RF_model):
    
    tree_predictions = []
    for j in np.arange(n_estimators):
        tree_predictions.append((RF_model.estimators_[j].predict(np.array([X]))).tolist())
    mean = np.mean(np.array(tree_predictions), axis=0)[0]

    
    std = np.std(np.array(tree_predictions), axis=0)[0]

    
    return 2 * std - mean

def MPI(X, RF_model, y_best):
    
    tree_predictions = []
    for j in np.arange(n_estimators):
        tree_predictions.append((RF_model.estimators_[j].predict(np.array([X]))).tolist())
    mean = np.mean(np.array(tree_predictions), axis=0)[0]
    
    std = np.std(np.array(tree_predictions), axis=0)[0]

    
    z = (y_best - mean)/std
    return norm.cdf(z)

def calc_MAE(test_index, RF_model):
    total_mae = 0
    for index in test_index:
        X = X_feature[index]
        y = X_instability[index]
        
        tree_predictions = []
        for j in np.arange(n_estimators):
            tree_predictions.append((RF_model.estimators_[j].predict(np.array([X]))).tolist())
        mean = np.mean(np.array(tree_predictions), axis=0)[0]

        std = np.std(np.array(tree_predictions), axis=0)[0]
        
        mae_ = (y - mean)**2
        total_mae += mae_
    return total_mae / len(test_index)

In [85]:
index_collection = []
X_collection = []
Instability_collection = []
InTopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    indices = list(np.arange(len(X_run)))
    
    top5_index = top5_indices
    X_top5 = []
    Instability_top5 = []
    for i in top5_index:
        X_top5.append(X_feature[i])
        Instability_top5.append(X_instability[i])
    
    index_ = random.sample(indices, n_initial)
    X_ = []
    Instability_ = []
    InTopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        Instability_.append(X_instability[i])
        indices.remove(i)
    
    test_index = random.sample(indices, n_test)
    X_test = []
    Instability_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        Instability_test.append(X_instability[i])
        indices.remove(i)
    
    n_opt = len(indices)
    assert n_opt == len(X_run) - n_test - n_initial
    
    for i in np.arange(n_opt):
        
        input_best = np.min(Instability_)
        
        RF_model = RandomForestRegressor(n_estimators=50)
        RF_model.fit(X_, Instability_)
        
        
        next_index = None
        max_ac = -10**10
        for j in indices:
            X_j = X_feature[j]
            Instability_j = X_instability[j]
            
            ac_value = EI(X_j, RF_model, input_best)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        Instability_.append(X_instability[next_index])
        indices.remove(next_index)
        index_.append(next_index)
        mae = calc_MAE(test_index, RF_model)
        MAE_.append(mae)
        how_many = Top5Percent(top5_index, index_)
        InTopPercent_.append(how_many)
        
#         print('Finished '+str(len(MAE_)) + ' round')
        
    assert len(MAE_) == len(X_) - n_initial

    index_collection.append(index_)
    X_collection.append(X_)
    Instability_collection.append(Instability_)
    InTopPercent_collection.append(InTopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, Instability_collection, InTopPercent_collection, MAE_collection])
np.save('RF_EI_master', master)

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed


In [86]:
index_collection = []
X_collection = []
Instability_collection = []
InTopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    indices = list(np.arange(len(X_run)))
    
    top5_index = top5_indices
    X_top5 = []
    Instability_top5 = []
    for i in top5_index:
        X_top5.append(X_feature[i])
        Instability_top5.append(X_instability[i])
    
    index_ = random.sample(indices, n_initial)
    X_ = []
    Instability_ = []
    InTopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        Instability_.append(X_instability[i])
        indices.remove(i)
    
    test_index = random.sample(indices, n_test)
    X_test = []
    Instability_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        Instability_test.append(X_instability[i])
        indices.remove(i)
    
    n_opt = len(indices)
    assert n_opt == len(X_run) - n_test - n_initial
    
    for i in np.arange(n_opt):
        
        input_best = np.min(Instability_)
        
        RF_model = RandomForestRegressor(n_estimators=50)
        RF_model.fit(X_, Instability_)
        
        
        next_index = None
        max_ac = -10**10
        for j in indices:
            X_j = X_feature[j]
            Instability_j = X_instability[j]
            
            ac_value = LCB(X_j, RF_model)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        Instability_.append(X_instability[next_index])
        indices.remove(next_index)
        index_.append(next_index)
        mae = calc_MAE(test_index, RF_model)
        MAE_.append(mae)
        how_many = Top5Percent(top5_index, index_)
        InTopPercent_.append(how_many)
        
#         print('Finished '+str(len(MAE_)) + ' round')
        
    assert len(MAE_) == len(X_) - n_initial

    index_collection.append(index_)
    X_collection.append(X_)
    Instability_collection.append(Instability_)
    InTopPercent_collection.append(InTopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, Instability_collection, InTopPercent_collection, MAE_collection])
np.save('RF_LCB_master', master)

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed


In [87]:
index_collection = []
X_collection = []
Instability_collection = []
InTopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    indices = list(np.arange(len(X_run)))
    
    top5_index = top5_indices
    X_top5 = []
    Instability_top5 = []
    for i in top5_index:
        X_top5.append(X_feature[i])
        Instability_top5.append(X_instability[i])
    
    index_ = random.sample(indices, n_initial)
    X_ = []
    Instability_ = []
    InTopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        Instability_.append(X_instability[i])
        indices.remove(i)
    
    test_index = random.sample(indices, n_test)
    X_test = []
    Instability_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        Instability_test.append(X_instability[i])
        indices.remove(i)
    
    n_opt = len(indices)
    assert n_opt == len(X_run) - n_test - n_initial
    
    for i in np.arange(n_opt):
        
        input_best = np.min(Instability_)
        
        RF_model = RandomForestRegressor(n_estimators=50)
        RF_model.fit(X_, Instability_)
        
        
        next_index = None
        max_ac = -10**10
        for j in indices:
            X_j = X_feature[j]
            Instability_j = X_instability[j]
            
            ac_value = MPI(X_j, RF_model, input_best)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        Instability_.append(X_instability[next_index])
        indices.remove(next_index)
        index_.append(next_index)
        mae = calc_MAE(test_index, RF_model)
        MAE_.append(mae)
        how_many = Top5Percent(top5_index, index_)
        InTopPercent_.append(how_many)
        
#         print('Finished '+str(len(MAE_)) + ' round')
        
    assert len(MAE_) == len(X_) - n_initial

    index_collection.append(index_)
    X_collection.append(X_)
    Instability_collection.append(Instability_)
    InTopPercent_collection.append(InTopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, Instability_collection, InTopPercent_collection, MAE_collection])
np.save('RF_MPI_master', master)

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed


In [18]:

RF_model = RandomForestRegressor(n_estimators=n_estimators)
RF_model.fit(X_feature, X_instability)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=50,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [43]:
np.array([X_feature[45]])

array([[0.27, 0.01, 0.72]])

In [75]:
RF_model.predict(np.array([X_feature[3]]))

array([380537.95])

In [76]:
tree_predictions = []
for j in np.arange(n_estimators):
    tree_predictions.append((RF_model.estimators_[j].predict(np.array([X_feature[45]]))).tolist())
np.mean(np.array(tree_predictions), axis = 0)

array([162359.614])

In [77]:
np.std(np.array(tree_predictions), axis=0)

array([150492.10910381])

In [84]:
EI(X_feature[0], RF_model, 15)

419.1159429616732

In [54]:
RF_model.predict(np.array([X_feature[45]]))[0]

221526.55

In [ ]:
# from sklearn.gaussian_process import GaussianProcessRegressor
# from sklearn.gaussian_process.kernels import (RBF, Matern, RationalQuadratic,
#                                               ExpSineSquared, DotProduct,
#                                               ConstantKernel)

# kernels = [1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 10.0)),
#            1.0 * RationalQuadratic(length_scale=1.0, alpha=0.1),
#            1.0 * ExpSineSquared(length_scale=1.0, periodicity=3.0,length_scale_bounds=(0.1, 10.0), periodicity_bounds=(1.0, 10.0)),
#            ConstantKernel(0.1, (0.01, 10.0))* (DotProduct(sigma_0=1.0, sigma_0_bounds=(0.1, 10.0)) ** 2),
#            1.0 * Matern(length_scale=1.0, length_scale_bounds=(1e-1, 10.0), nu=1.5)]

# model = GaussianProcessRegressor(alpha = 1e-10, 
#                                  kernel = kernels[0],
#                                  optimizer = 'fmin_l_bfgs_b', 
#                                  n_restarts_optimizer = 1, 
#                                  random_state=0,
#                                  normalize_y=True)

# # feature order: ['CsPbI', 'MAPbI', 'FAPbI']
# model.fit(X_feature, X_instability)

# Why these models from sklearn don't work well, I'm confused...